# Script: Beta Diversity + db-RDA + Visualization

## Description

This script performs an integrated ecological analysis of microbial community structure and environmental factors, using:
- Bray-Curtis beta diversity and null model SES
- PCoA ordination
- Distance-based Redundancy Analysis (db-RDA)
- Variation partitioning (environment vs. stochastic)
- Multiple plots for interpretation

## Key Outputs

- `Within_Group_Beta_Diversity.png`: Boxplot of within-group dissimilarities
- `Variation_Contribution_Fixed.png`: Partitioning of explained vs. random variation
- `PCoA_Scatter.png`: PCoA with group colors
- `PCoA_With_Ellipses.png`: PCoA with confidence ellipses
- `Environmental_Correlation.png`: Correlation heatmap of environmental factors
- `Beta_Diversity_Heatmap.png`: Heatmap of pairwise beta diversity

## Requirements

- Python 3.8+
- Packages: pandas, numpy, matplotlib, seaborn, scikit-bio, statsmodels, sklearn


## 📊 Integrated Beta Diversity and db-RDA Analysis

This module computes null-model corrected beta diversity, runs ordination (PCoA), performs distance-based RDA (db-RDA), and visualizes how environmental factors explain community variation.

### Key Features

- Bray-Curtis dissimilarity matrix
- Null model simulation and SES calculation
- db-RDA variance partitioning
- Rich plots: boxplots, PCoA with ellipses, heatmaps

### Run

```bash
python scripts/beta_diversity_dbrda_plot.py


In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from skbio.diversity import beta_diversity
from skbio.stats.ordination import pcoa
from skbio.stats.distance import DistanceMatrix
from scipy.stats import ttest_1samp
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
import seaborn as sns

# File paths
relative_abundance_file = r'.../grelative.xls.csv'
env_factors_file = r'.../Enviroment elements B.csv'
output_folder = r'.../BetaDiversityResults'
os.makedirs(output_folder, exist_ok=True)

# Load OTU and environmental data
otu_df = pd.read_csv(relative_abundance_file, index_col=0)
env_factors = pd.read_csv(env_factors_file, index_col=0)

# Align samples between OTU and environmental data
common_samples = otu_df.columns.intersection(env_factors.index)
otu_df = otu_df[common_samples]
env_factors = env_factors.loc[common_samples]

# Normalize OTU relative abundance
otu_df = otu_df.div(otu_df.sum(axis=0), axis=1)

# Function: calculate null-model-corrected beta diversity and SES
def calculate_beta_diversity_with_null_model(data, metric="braycurtis", num_iterations=100):
    observed_dist = beta_diversity(metric, data.values.T, ids=data.columns)
    observed_values = observed_dist.to_data_frame().values.flatten()

    null_distributions = []
    for _ in range(num_iterations):
        randomized_data = data.apply(np.random.permutation, axis=0)
        null_dist = beta_diversity(metric, randomized_data.values.T, ids=randomized_data.columns)
        null_distributions.append(null_dist.to_data_frame().values.flatten())

    null_distributions = pd.DataFrame(null_distributions)
    null_mean = null_distributions.mean(axis=1).mean()
    null_std = null_distributions.values.flatten().std()
    observed_mean = observed_values.mean()
    ses = (observed_mean - null_mean) / null_std

    t_stat, p_value = ttest_1samp(null_distributions.mean(axis=1), observed_mean)
    return observed_dist, ses, p_value

# Calculate beta diversity and SES
observed_dist, ses, p_value = calculate_beta_diversity_with_null_model(otu_df)
print(f"SES: {ses:.3f}, p-value: {p_value:.3f}")

# Perform PCoA
distance_matrix = DistanceMatrix(observed_dist.to_data_frame().values, ids=otu_df.columns)
pcoa_results = pcoa(distance_matrix)

# Ensure matching samples
common_samples = list(set(distance_matrix.ids).intersection(env_factors.index))
if not common_samples:
    raise ValueError("No overlapping samples between distance matrix and environmental data.")

# Function: db-RDA analysis
def perform_dbrda(distance_matrix, env_data):
    pcoa_axes = pcoa(distance_matrix).samples

    common_samples = list(set(distance_matrix.ids).intersection(env_data.index))
    pcoa_axes = pcoa_axes.loc[common_samples]
    env_data_dummies = pd.get_dummies(env_data.loc[common_samples], drop_first=True)

    env_data_dummies = env_data_dummies.loc[:, env_data_dummies.std() > 0]
    if env_data_dummies.shape[1] == 0:
        raise ValueError("No valid columns in environmental data.")

    # Standardize environmental variables
    scaler = StandardScaler()
    env_data_dummies = pd.DataFrame(scaler.fit_transform(env_data_dummies), 
                                    index=env_data_dummies.index, 
                                    columns=env_data_dummies.columns)

    if not (pcoa_axes.index.equals(env_data_dummies.index)):
        raise ValueError("Sample indices do not match.")

    results = []
    for i in range(pcoa_axes.shape[1]):
        y = pcoa_axes.iloc[:, i]
        model = sm.OLS(y, sm.add_constant(env_data_dummies)).fit()
        results.append(model.rsquared)

    R2 = np.mean(results)
    R2_adj = 1 - (1 - R2) * (len(common_samples) - 1) / (len(common_samples) - env_data_dummies.shape[1] - 1)
    return R2, R2_adj

# Run db-RDA
try:
    R2, R2_adj = perform_dbrda(distance_matrix, env_factors)
    print(f"R²: {R2:.3f}, Adjusted R²: {R2_adj:.3f}")
except ValueError as e:
    print(f"db-RDA error: {e}")

# Plot within-group beta diversity
observed_df_long = observed_dist.to_data_frame().stack().reset_index()
observed_df_long.columns = ['Sample1', 'Sample2', 'Distance']
observed_df_long['Group1'] = observed_df_long['Sample1'].map(env_factors['Label'])
observed_df_long['Group2'] = observed_df_long['Sample2'].map(env_factors['Label'])
observed_df_long = observed_df_long[observed_df_long['Group1'] == observed_df_long['Group2']]

plt.figure(figsize=(10, 6))
sns.boxplot(data=observed_df_long, x='Group1', y='Distance', palette='Set2')
plt.title('Within-group β Diversity')
plt.ylabel('Bray-Curtis Distance')
plt.xlabel('Group')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(output_folder, 'Within_Group_Beta_Diversity.png'))
plt.show()

# Plot variation partitioning
if R2_adj is None or np.isnan(R2_adj) or R2_adj < 0:
    R2_adj = 0
random_contribution = 1 - R2_adj

plt.figure(figsize=(6, 4))
plt.bar(["Environment", "Random"], [R2_adj, random_contribution], color=["green", "orange"])
plt.title("Variation Partitioning")
plt.ylabel("Proportion Explained")
plt.ylim(0, 1)
plt.tight_layout()
plt.savefig("Variation_Contribution_Fixed.png")
plt.show()

# PCoA scatter plot by group
plt.figure(figsize=(10, 8))
for group, color in zip(env_factors['Label'].unique(), sns.color_palette('Set1', n_colors=len(env_factors['Label'].unique()))):
    subset = pcoa_results.samples[env_factors['Label'] == group]
    plt.scatter(subset.iloc[:, 0], subset.iloc[:, 1], label=group, color=color)

plt.title('PCoA Analysis')
plt.xlabel('PC1 ({}%)'.format(round(pcoa_results.proportion_explained[0] * 100, 2)))
plt.ylabel('PC2 ({}%)'.format(round(pcoa_results.proportion_explained[1] * 100, 2)))
plt.legend(title='Group')
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(output_folder, 'PCoA_Scatter.png'))
plt.show()

# PCoA with ellipses
plt.figure(figsize=(10, 8))
sns.scatterplot(x=pcoa_results.samples.iloc[:, 0], y=pcoa_results.samples.iloc[:, 1], 
                hue=env_factors['Label'], palette='Set2', s=100)
for group in env_factors['Label'].unique():
    subset = pcoa_results.samples[env_factors['Label'] == group]
    sns.kdeplot(x=subset.iloc[:, 0], y=subset.iloc[:, 1], levels=2, color='gray', alpha=0.5)

plt.title('PCoA with Group Ellipses')
plt.xlabel('PC1 ({}%)'.format(round(pcoa_results.proportion_explained[0] * 100, 2)))
plt.ylabel('PC2 ({}%)'.format(round(pcoa_results.proportion_explained[1] * 100, 2)))
plt.legend(title='Group')
plt.tight_layout()
plt.savefig(os.path.join(output_folder, 'PCoA_With_Ellipses.png'))
plt.show()

# Environmental factor correlation heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(env_factors.corr(), annot=True, cmap='coolwarm')
plt.title('Environmental Factors Correlation')
plt.tight_layout()
plt.savefig(os.path.join(output_folder, 'Environmental_Correlation.png'))
plt.show()

# Beta diversity heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(observed_dist.to_data_frame(), cmap='coolwarm', xticklabels=True, yticklabels=True)
plt.title('β Diversity Heatmap')
plt.savefig(os.path.join(output_folder, 'Beta_Diversity_Heatmap.png'))
plt.show()
